In [1]:
import tensorflow as tf
from tensorflow import keras
#print(tf.keras.__version__)
#print(tf.__version__)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math as mt
import random as rng

from sklearn import preprocessing

def getColumns(size):
    columns = []
    for i in range(size):
        columns.append(i)
    return columns

directory = "./CMAPSSData/train_FD001.txt"
turbo = pd.read_csv(directory, header=None, sep=' ')
turbo = turbo.drop([26, 27], axis=1)

#for i in turbo.columns:
#    column = turbo[i]
#    print(i)
#    print(column.describe())
    
#print(turbo.values)

# Drop the input parameters to get just the sensors
turboSens = turbo.drop([0,1,4], axis=1)
turboSens.columns = getColumns(turboSens.shape[1])

# Get the cycles and useful sensors. 
turboUsefulSens = turboSens.drop([0, 9, 17, 18], axis=1)

min_max_scaler = preprocessing.MinMaxScaler()
norm_sens = min_max_scaler.fit_transform(turboUsefulSens)

turboCycles = turbo[1]
#print(turboCycles)

deteriorate = []
maxCycle = float(turboCycles.max())


currentCount = maxCycle
for i in range(turbo.shape[0] - 1, -1, -1):
    currentRow = turboCycles.loc[i]
    deteriorate.insert(0, currentCount)
    if currentRow == 1:
        currentCount = maxCycle
    else:
        currentCount -= 1

        
deteriorate = pd.DataFrame(deteriorate)

deteriorateSwap = []
for i in deteriorate[0]:
    deteriorateSwap.append(maxCycle - i)
deteriorate = pd.DataFrame(deteriorateSwap)


norm_sens = pd.DataFrame(norm_sens)
deteriorate = deteriorate.join(norm_sens, lsuffix='0', rsuffix='1')
deteriorate.columns = getColumns(deteriorate.shape[1])



In [2]:
array = deteriorate.to_numpy()

arrayCopy = array.tolist()

simulations = []

convCount = 10
convDiv = int(convCount/2)

count = len(array)

while(count > 0):
    cycles = []
    element = [-1,-1]
    while(element[0]!=0):
        element = arrayCopy.pop(0)
        cycles.append(element)
        count-=1
    simulations.append(cycles)
    
#for sim in simulations:
#    lenth = len(sim)
#    remainder = lenth%convCount
#    for i in range(remainder):
#        sim.pop(0)
    
#simsRecomb = []
#for sim in simulations:
#    for i in sim:
#        simsRecomb.append(i)

#print(len(simsRecomb))

In [3]:

tens = []
tensCount = []

for sim in simulations:
    for i in range(convDiv, len(sim) - convDiv):
        ten = []
        total = 0
        for j in range(-convDiv,convDiv):
            index = i + j
            element = sim[index].copy()
            total+= element.pop(0)
            ten.append(element)
        total/=convCount
        tensCount.append(total)
        tens.append(ten)

    
print(len(tens))
print(len(tensCount))

19631
19631


In [4]:
simsReorder = []
simsCycleReorder = []

length = len(tens)
for i in range(length):
    index = rng.randint(0, len(tens)-1)
    simsReorder.append(tens.pop(index))
    simsCycleReorder.append(tensCount.pop(index))
    
    
    
simsTrain = simsReorder[:15704]
targetTrain = simsCycleReorder[:15704]

simsTest = simsReorder[15704:]
targetTest = simsCycleReorder[15704:]


width = (turboUsefulSens.shape[1])

In [5]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(150, 3, activation='relu', input_shape=(convCount, width)),
    tf.keras.layers.Conv1D(300, 3, activation='relu'),
    tf.keras.layers.MaxPooling1D(6),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 8, 150)            8700      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 6, 300)            135300    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1, 300)            0         
_________________________________________________________________
dense (Dense)                (None, 1, 128)            38528     
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              129       
Total params: 182,657
Trainable params: 182,657
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='mse', 
              metrics=['mae', 'mse'])

In [7]:
model.fit(simsTrain, targetTrain, batch_size=16, epochs=100, verbose=1)

Train on 15704 samples
Epoch 1/100
15704/15704 [==============================] - 4s 251us/sample - loss: 2144.9349 - mae: 33.9990 - mse: 2144.9355
Epoch 2/100
15704/15704 [==============================] - 3s 192us/sample - loss: 1753.2890 - mae: 30.1392 - mse: 1753.2891
Epoch 3/100
15704/15704 [==============================] - 3s 193us/sample - loss: 1692.3279 - mae: 29.3145 - mse: 1692.3287
Epoch 4/100
15704/15704 [==============================] - 3s 193us/sample - loss: 1662.0943 - mae: 28.9248 - mse: 1662.0950
Epoch 5/100
15704/15704 [==============================] - 3s 188us/sample - loss: 1636.3607 - mae: 28.6433 - mse: 1636.3600
Epoch 6/100
15704/15704 [==============================] - 3s 189us/sample - loss: 1612.0837 - mae: 28.4362 - mse: 1612.0837
Epoch 7/100
15704/15704 [==============================] - 3s 192us/sample - loss: 1582.7211 - mae: 27.9988 - mse: 1582.7217
Epoch 8/100
15704/15704 [==============================] - 3s 187us/sample - loss: 1573.2594 - mae: 28

In [8]:
model.evaluate(simsTest,targetTest)

3927/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[206.46935421812768, 10.425427, 206.46938]

In [12]:


print(model.predict([simsTest[0]]))
 
print(targetTest[0])

print(model.predict([simsTest[1]]))

print(targetTest[1])

[[[152.94646]]]
161.5
[[[108.17484]]]
106.5
